In [1]:
import pandas as pd
import numpy as np
from datetime import date
import unidecode

In [2]:
ls

deputies.json                Leg_2017_Resultats_T2_c.numbers  Untitled.ipynb
french.json*                 Leg_2017_Resultats_T2_c.xlsx
Leg_2017_Resultats_T2_c.csv  liste.csv*


In [3]:
res = pd.read_csv("Leg_2017_Resultats_T2_c.csv", sep=";")
htpr = pd.read_csv("liste.csv")

## Préparation de la table de résultat

In [4]:
res = res.rename(columns={"Né(e)_le":"date"})

In [5]:
def calculate_age(born):
    today = date.today()
    born = born.split("/")
    born_dict = {"year": int("19"+born[2]) ,"day":int(born[1]),"month":int(born[0]) }
    return today.year - born_dict["year"] - ((today.month, today.day) < (born_dict["month"], born_dict["day"]))

In [6]:
calculate_age(res["date"][0])

43

In [7]:
res["age"] = res["date"].map(lambda x : calculate_age(x))

In [8]:
res["nom"] = res["Désignation"].map(lambda x: unidecode.unidecode(x.split(" ")[1].strip()).upper())
res["prenom"] = res["Désignation"].map(lambda x:unidecode.unidecode(x.split(" ")[2].strip()).upper())

In [9]:
res["sexe"] = res["Désignation"].map(lambda x : "M" if x.split(" ")[0]=="M." else "F")

In [10]:
res.head()

,Dpt,Nom_de_la_Circonscription,Désignation,Nuance,Score,Sortant,Perso.,date,Profession,Tour,age,nom,prenom,sexe
0,ZA,1ère circonscription,M. SERVA Olivier,REM,"61,74 %",NaN,NaN,6/21/74,34 - Expert comptable,2,43,SERVA,OLIVIER,M
1,ZA,2ème circonscription,Mme BENIN Justine,DVG,"64,26 %",NaN,NaN,3/12/75,48 - Fonctionnaire de catégorie B,2,42,BENIN,JUSTINE,F
2,ZA,3ème circonscription,M. MATHIASIN Max,DVG,"65,15 %",NaN,NaN,2/24/56,52 - Employé (autres entrep. publiques),2,61,MATHIASIN,MAX,M
3,ZA,4ème circonscription,Mme VAINQUEUR-CHRISTOPHE Hélène,SOC,"61,61 %",NaN,NaN,5/6/56,28 - Pharmacien,2,61,VAINQUEUR-CHRISTOPHE,HELENE,F
4,ZB,1ère circonscription,Mme MANIN Josette,DVG,"54,89 %",NaN,NaN,3/16/50,61 - Retraité salarié privé,2,67,MANIN,JOSETTE,F


## Préparation de la table de la haute autorité

In [11]:
htpr["nom"] = htpr["nom"].map(lambda x:unidecode.unidecode(x.strip()).upper())
htpr["prenom"] = htpr["prenom"].map(lambda x:unidecode.unidecode(x.strip()).upper())

In [12]:
htpr.head()

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,nom_fichier,url_dossier,id_origine,url_photo
0,Mme,ABBASSIA,HAKEM,hakemaaaabbassia4615,commune,Adjointe au maire de Nantes,di,44,2016-07-07,hakem-abbassia-di-commune-nantes.pdf,/pages_nominatives/hakem-abbassia,NaN,NaN
1,M.,ABDEL,GHEZALI,ghezaliaaaabdel3656,commune,Adjoint au maire de Besançon,di,25,2016-07-07,ghezali-abdel-di-commune-besancon.pdf,/pages_nominatives/ghezali-abdel,NaN,NaN
2,M.,ABDEL-MADJID,SADI,sadiaaaabdelmadjid8238,departement,Vice-Président du conseil départemental de la ...,di,93,2016-10-28,sadi-abdel-madjid-di-departement-93.pdf,/pages_nominatives/sadi-abdel-madjid-8238,NaN,NaN
3,M.,ABDOURAHAMANE,SOILIHI,soilihiaaaabdourahamane8213,senateur,Sénateur de Mayotte,dia,976,2014-07-23,soilihi-abdourahamane-dia-senateur-976.pdf,/pages_nominatives/soilihi-abdourahamane,11071M,http://www.senat.fr/senimg/soilihi_abdourahama...
4,M.,ABDOURAHAMANE,SOILIHI,soilihiaaaabdourahamane8213,senateur,Sénateur de Mayotte,dsp,976,2015-07-10,soilihi-abdourahamane-dsp-senateur-976.pdf,/pages_nominatives/soilihi-abdourahamane,11071M,http://www.senat.fr/senimg/soilihi_abdourahama...


In [13]:
htpr[htpr["nom"]=="BAYROU"]

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,nom_fichier,url_dossier,id_origine,url_photo
1770,M.,FRANCOIS,BAYROU,bayrouaaafrancois58,commune,Maire de Pau,di,64,2015-12-17,bayrou-francois-di-commune-pau.pdf,/pages_nominatives/bayrou-francois,NaN,NaN


In [14]:
# Garde que les personnes qui ont un dossier renseignée 
htpr = htpr[~htpr["url_dossier"].isnull()]
htpr["link"] = htpr["nom_fichier"].map(lambda x : "http://www.hatvp.fr/livraison/dossiers/"+str(x))

In [15]:
print(htpr.shape)

(5563, 14)


In [17]:
# Prendre la photo la plus récente 
sub_htrp = htpr.dropna()
sub_htrp = sub_htrp.sort_values(["nom","prenom","date_publication"], ascending=False)
sub_htrp = sub_htrp.drop_duplicates(["nom","prenom"])
sub_htrp = sub_htrp.rename(columns={"url_photo": "link_pict"})

In [18]:
htpr = pd.merge(htpr, sub_htrp[["nom","prenom","link_pict"]],how="left", on=["nom","prenom"])

In [19]:
htpr.head()

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,nom_fichier,url_dossier,id_origine,url_photo,link,link_pict
0,Mme,ABBASSIA,HAKEM,hakemaaaabbassia4615,commune,Adjointe au maire de Nantes,di,44,2016-07-07,hakem-abbassia-di-commune-nantes.pdf,/pages_nominatives/hakem-abbassia,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/hakem-a...,NaN
1,M.,ABDEL,GHEZALI,ghezaliaaaabdel3656,commune,Adjoint au maire de Besançon,di,25,2016-07-07,ghezali-abdel-di-commune-besancon.pdf,/pages_nominatives/ghezali-abdel,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/ghezali...,NaN
2,M.,ABDEL-MADJID,SADI,sadiaaaabdelmadjid8238,departement,Vice-Président du conseil départemental de la ...,di,93,2016-10-28,sadi-abdel-madjid-di-departement-93.pdf,/pages_nominatives/sadi-abdel-madjid-8238,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/sadi-ab...,NaN
3,M.,ABDOURAHAMANE,SOILIHI,soilihiaaaabdourahamane8213,senateur,Sénateur de Mayotte,dia,976,2014-07-23,soilihi-abdourahamane-dia-senateur-976.pdf,/pages_nominatives/soilihi-abdourahamane,11071M,http://www.senat.fr/senimg/soilihi_abdourahama...,http://www.hatvp.fr/livraison/dossiers/soilihi...,http://www.senat.fr/senimg/soilihi_abdourahama...
4,M.,ABDOURAHAMANE,SOILIHI,soilihiaaaabdourahamane8213,senateur,Sénateur de Mayotte,dsp,976,2015-07-10,soilihi-abdourahamane-dsp-senateur-976.pdf,/pages_nominatives/soilihi-abdourahamane,11071M,http://www.senat.fr/senimg/soilihi_abdourahama...,http://www.hatvp.fr/livraison/dossiers/soilihi...,http://www.senat.fr/senimg/soilihi_abdourahama...


In [20]:
print(htpr.shape)

(5563, 15)


In [21]:
# garder la déclaration la plus récente 
htpr = htpr.sort_values(["nom","prenom","date_publication"], ascending=False)

In [23]:
htpr.head()

,civilite,prenom,nom,classement,type_mandat,qualite,type_document,departement,date_publication,nom_fichier,url_dossier,id_origine,url_photo,link,link_pict
589,M.,BERNARD,ZUNINO,zuninoaaabernard9220,commune,Maire de Saint-Michel-sur-Orge,di,91,2015-12-17,zunino-bernard-di-commune-saint-michel-sur-org...,/pages_nominatives/zunino-bernard,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/zunino-...,NaN
4090,M.,MICHEL,ZUMKELLER,zumkelleraaamichel9238,depute,Député du Territoire de Belfort,dsp,90,2015-07-10,zumkeller-michel-dsp-depute-90.pdf,/pages_nominatives/zumkeller-michel,267330,http://www2.assemblee-nationale.fr/static/trib...,http://www.hatvp.fr/livraison/dossiers/zumkell...,http://www2.assemblee-nationale.fr/static/trib...
4089,M.,MICHEL,ZUMKELLER,zumkelleraaamichel9238,depute,Député du Territoire de Belfort,dia,90,2014-07-23,zumkeller-michel-dia-depute-90.pdf,/pages_nominatives/zumkeller-michel,267330,http://www2.assemblee-nationale.fr/static/trib...,http://www.hatvp.fr/livraison/dossiers/zumkell...,http://www2.assemblee-nationale.fr/static/trib...
3764,Mme,MARIE,ZUCCARELLI,ZUCCARELLI Marie10371,departement,Conseillère départementale de la Corse-du-Sud,di,2A,NaN,NaN,/pages_nominatives/zuccarelli-marie,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/nan,NaN
2933,M.,JEAN-MARIE,ZOELLE,zoelleaaajeanmarie9213,commune,Maire de Saint-Louis,di,57,2015-12-17,zoelle-jean-marie-di-commune-saint-louis.pdf,/pages_nominatives/zoelle-jean-marie,NaN,NaN,http://www.hatvp.fr/livraison/dossiers/zoelle-...,NaN


In [24]:
htpr = htpr.drop_duplicates(["nom","prenom"], keep='first')

In [26]:
htpr.shape

(3417, 15)

## Préparation table résultat premier tour 

In [51]:
vrai_res = pd.read_excel("Leg_2017_Resultats_T1_c.xlsx", sheetname="Circo. leg. T1", skiprows=2)

In [52]:
vrai_res["code"] = vrai_res.apply(lambda row : str(row["Code du département"])+'|'
                                  +str(row["Code de la circonscription"]), axis=1)

vrai_res = vrai_res.set_index("code")

one_df = vrai_res[["Nom","Nom.1","Nom.2","Nom.3","Nom.4","Nom.5","Nom.6",
         "Nom.7","Nom.8","Nom.9","Nom.10","Nom.11","Nom.12","Nom.13","Nom.14","Nom.15","Nom.16","Nom.17","Nom.18",
         "Nom.19","Nom.20","Nom.21","Nom.22","Nom.23","Nom.24",
         "Nom.25","Nom.26"]].stack()
one_df = one_df.rename("nom")
one_df = one_df.reset_index()
del one_df["level_1"]

two_df = vrai_res[["Prénom","Prénom.1","Prénom.2","Prénom.3","Prénom.4","Prénom.5","Prénom.6",
         "Prénom.7","Prénom.8","Prénom.9","Prénom.10","Prénom.11","Prénom.12","Prénom.13","Prénom.14",
        "Prénom.15","Prénom.16","Prénom.17","Prénom.18",
         "Prénom.19","Prénom.20","Prénom.21","Prénom.22","Prénom.23","Prénom.24",
         "Prénom.25","Prénom.26"]].stack()
two_df = two_df.rename("prenom")
two_df = two_df.reset_index()
del two_df["code"]
del two_df["level_1"]

tree_df = vrai_res[["% Voix/Exp","% Voix/Exp.1","% Voix/Exp.2","% Voix/Exp.3","% Voix/Exp.4","% Voix/Exp.5",
                    "% Voix/Exp.6",
         "% Voix/Exp.7","% Voix/Exp.8","% Voix/Exp.9","% Voix/Exp.10","% Voix/Exp.11","% Voix/Exp.12",
                    "% Voix/Exp.13","% Voix/Exp.14",
        "% Voix/Exp.15","% Voix/Exp.16","% Voix/Exp.17","% Voix/Exp.18",
         "% Voix/Exp.19","% Voix/Exp.20","% Voix/Exp.21","% Voix/Exp.22","% Voix/Exp.23","% Voix/Exp.24",
         "% Voix/Exp.25","% Voix/Exp.26"]].stack()
tree_df = tree_df.rename("score_exp")
tree_df = tree_df.reset_index()
del tree_df["code"]
del tree_df["level_1"]


quar_df = vrai_res[["% Voix/Ins","% Voix/Ins.1","% Voix/Ins.2","% Voix/Ins.3","% Voix/Ins.4","% Voix/Ins.5",
                    "% Voix/Ins.6",
         "% Voix/Ins.7","% Voix/Ins.8","% Voix/Ins.9","% Voix/Ins.10","% Voix/Ins.11","% Voix/Ins.12",
                    "% Voix/Ins.13","% Voix/Ins.14",
        "% Voix/Ins.15","% Voix/Ins.16","% Voix/Ins.17","% Voix/Ins.18",
         "% Voix/Ins.19","% Voix/Ins.20","% Voix/Ins.21","% Voix/Ins.22","% Voix/Ins.23","% Voix/Ins.24",
         "% Voix/Ins.25","% Voix/Ins.26"]].stack()
quar_df = quar_df.rename("score_ins")
quar_df = quar_df.reset_index()
del quar_df["code"]
del quar_df["level_1"]

df_resultat = pd.concat([one_df, two_df, tree_df, quar_df], axis=1)

In [53]:
df_resultat["nom"] = df_resultat["nom"].map(lambda x: unidecode.unidecode(x.strip()).upper())
df_resultat["prenom"] = df_resultat["prenom"].map(lambda x: unidecode.unidecode(x.strip()).upper())

In [54]:
df_resultat.head()

,code,nom,prenom,score_exp,score_ins
0,1|1,LEPAGNOT,MAUDE,0.73,0.35
1,1|1,FONTAINE,JACQUES,1.64,0.79
2,1|1,MARTIN ZEMLIK,FABRINE,9.70,4.68
3,1|1,BLATRIX-CONTAT,FLORENCE,9.23,4.46
4,1|1,RAIMONDO,LAURANE,1.41,0.68


## Merge table 

In [27]:
print(res.shape)

(577, 14)


In [42]:
json = pd.merge(res, htpr[["nom","prenom","link","link_pict"]], how="left", on=["nom","prenom"])

In [55]:
# 40 NOMS qui déconne à régler
json = pd.merge(json, df_resultat, on=["nom","prenom"])

In [56]:
json.shape

(536, 19)